In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Flatten,Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd
import itertools
import shutil
import random
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [ ]:
Train="/content/drive/MyDrive/3classnew/Old"
Test= "/content/drive/MyDrive/3classnew/OldTest"
NewData="/content/drive/MyDrive/3classnew/NEw Data"

In [ ]:
import keras
def load_images(directory):
    images = []
    labels = []
    for idx, label in enumerate(uniq_labels):
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            image = cv2.resize(cv2.imread(filepath,cv2.IMREAD_COLOR), (224, 224))
            image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)
            images.append(image)
            labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return(images, labels)

uniq_labels = sorted(os.listdir(Train))
image, labels = load_images(directory = Train)
if uniq_labels == sorted(os.listdir(Test)):
  X_test, y_test = load_images(directory = Test)
if uniq_labels == sorted(os.listdir(NewData)):
  X_new, y_new = load_images(directory = NewData)

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(image,labels, test_size =0.2, stratify = labels)

In [ ]:
X_train = X_train.astype('float32')/255.0
X_eval= X_eval.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0
X_new = X_new.astype('float32')/255.0


In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_eval = to_categorical(y_eval)
y_new = to_categorical(y_new)

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=10,  
    width_shift_range=0.1,  
    height_shift_range=0.2, 
    zoom_range=0.10, 
     shear_range=0.1,
    horizontal_flip=False,
    fill_mode="nearest")


In [ ]:
Train_data=train_datagen.flow(X_train,y_train,batch_size=27,shuffle=True)
Evalu_data=train_datagen.flow(X_eval,y_eval,batch_size=27,shuffle=True)
Test_data=train_datagen.flow(X_test,y_test,batch_size=27,shuffle=True)
New_data= train_datagen.flow(X_new,y_new,batch_size=27,shuffle=True)


**Fine Tuning **

In [ ]:
VGG16_model = tf.keras.models.load_model('/content/drive/MyDrive/3classnew/VGG6_3c.h5')

In [ ]:
for layer in VGG16_model.layers:
    layer.trainable = True

In [ ]:
VGG16_model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.01)

In [ ]:
history1 = VGG16_model.fit(Train_data,epochs=5,batch_size= 1,validation_data=Evalu_data,callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=5,
                            restore_best_weights=True),

                        learning_rate_reduction
                    ])

Epoch 1/5
26/26 [==============================] - 46s 1s/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 3.8191e-04 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 2/5
26/26 [==============================] - 21s 807ms/step - loss: 0.0071 - accuracy: 0.9971 - val_loss: 1.5171e-04 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 3/5
26/26 [==============================] - 21s 812ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 2.4312e-05 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 4/5
26/26 [==============================] - 21s 813ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 1.8140e-04 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 5/5
26/26 [==============================] - 21s 809ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 9.5277e-06 - val_accuracy: 1.0000 - lr: 1.0000e-05


In [ ]:
VGG16_model.save('/content/drive/MyDrive/3classnew/FinVGG16_3c.h5')

**Testing the Model**

In [ ]:
score = VGG16_model.evaluate(Test_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = VGG16_model.evaluate(Evalu_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 100.0 %
Accuracy for evaluation images: 100.0 %


**Testing the model outside of data base**

In [ ]:
score = VGG16_model.evaluate(x= X_new, y = y_new, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = VGG16_model.evaluate(New_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 56.19 %
Accuracy for evaluation images: 55.714 %


**Inception V3** **Fine tuning**

In [ ]:
InceptionV3_model = tf.keras.models.load_model('/content/drive/MyDrive/3classnew/InceptionV3_3c.h5')

In [ ]:
for layer in InceptionV3_model.layers:
    layer.trainable = True

In [ ]:
InceptionV3_model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history3 = InceptionV3_model.fit(Train_data,epochs=5,batch_size= 1,validation_data=Evalu_data,callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=5,
                            restore_best_weights=True),

                        learning_rate_reduction
                    ])

Epoch 1/5
26/26 [==============================] - 33s 837ms/step - loss: 0.0661 - accuracy: 0.9812 - val_loss: 0.2376 - val_accuracy: 0.9480 - lr: 1.0000e-05
Epoch 2/5
26/26 [==============================] - 16s 604ms/step - loss: 0.0822 - accuracy: 0.9841 - val_loss: 0.1421 - val_accuracy: 0.9422 - lr: 1.0000e-05
Epoch 3/5
26/26 [==============================] - 16s 601ms/step - loss: 0.0286 - accuracy: 0.9928 - val_loss: 0.0980 - val_accuracy: 0.9538 - lr: 1.0000e-05
Epoch 4/5
26/26 [==============================] - 16s 604ms/step - loss: 0.0536 - accuracy: 0.9841 - val_loss: 0.0639 - val_accuracy: 0.9769 - lr: 1.0000e-05
Epoch 5/5
26/26 [==============================] - 16s 604ms/step - loss: 0.0620 - accuracy: 0.9884 - val_loss: 0.0400 - val_accuracy: 0.9769 - lr: 1.0000e-05


In [ ]:
InceptionV3_model.save('/content/drive/MyDrive/3classnew/FinIncep_3c.h5')

**Testing** **The** **model** **bold text**

In [ ]:
score = InceptionV3_model.evaluate(Test_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = InceptionV3_model.evaluate(Evalu_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 98.728 %
Accuracy for evaluation images: 97.11 %


Testing ouside of Data set

In [ ]:
score = InceptionV3_model.evaluate(x= X_new, y = y_new, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = InceptionV3_model.evaluate(New_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 46.19 %
Accuracy for evaluation images: 47.619 %


***Fine Tuning ResNet50***

In [ ]:
Resnet50_model = tf.keras.models.load_model('/content/drive/MyDrive/3classnew/Resnet_3c.h5')

In [ ]:
for layer in Resnet50_model.layers:
    layer.trainable = True   
 

In [ ]:
Resnet50_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history14 = Resnet50_model.fit(Train_data,epochs=27,batch_size= 1,validation_data=Evalu_data,callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=5,
                            restore_best_weights=True),

                        learning_rate_reduction
                    ])

Epoch 1/27
26/26 [==============================] - 33s 844ms/step - loss: 0.6758 - accuracy: 0.8815 - val_loss: 0.4626 - val_accuracy: 0.8439 - lr: 1.0000e-04
Epoch 2/27
26/26 [==============================] - 18s 680ms/step - loss: 0.0424 - accuracy: 0.9899 - val_loss: 0.6956 - val_accuracy: 0.7283 - lr: 1.0000e-04
Epoch 3/27
26/26 [==============================] - 18s 683ms/step - loss: 0.0223 - accuracy: 0.9942 - val_loss: 2.0132 - val_accuracy: 0.3988 - lr: 1.0000e-04
Epoch 4/27
26/26 [==============================] - 18s 689ms/step - loss: 0.0223 - accuracy: 0.9913 - val_loss: 2.6691 - val_accuracy: 0.3988 - lr: 1.0000e-04
Epoch 5/27
26/26 [==============================] - 18s 698ms/step - loss: 0.0483 - accuracy: 0.9942 - val_loss: 2.5229 - val_accuracy: 0.3584 - lr: 1.0000e-04
Epoch 6/27
26/26 [==============================] - 18s 699ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 3.2203 - val_accuracy: 0.2659 - lr: 1.0000e-04


In [ ]:
Resnet50_model.save('/content/drive/MyDrive/3classnew/Resnet50New1_3c.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


***Testing the Model ***

In [ ]:
score = Resnet50_model.evaluate(Test_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = Resnet50_model.evaluate(Evalu_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 82.081 %
Accuracy for evaluation images: 78.613 %


In [ ]:
score = Resnet50_model.evaluate(Test_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = Resnet50_model.evaluate(Evalu_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 99.884 %
Accuracy for evaluation images: 100.0 %


Testing out side of the data test

In [ ]:
score = Resnet50_model.evaluate(x= X_new, y = y_new, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = Resnet50_model.evaluate(New_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 30.952 %
Accuracy for evaluation images: 30.0 %


**Fine Tuning MobileNetV2**

In [ ]:
MobileNetV2_Model=tf.keras.models.load_model('/content/drive/MyDrive/3classnew/modelNetV2_3c.h5')

In [ ]:
for layer in MobileNetV2_Model.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = False
    else:
      layer.trainable = True    

In [ ]:
MobileNetV2_Model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history24 = MobileNetV2_Model.fit(Train_data,epochs=27,batch_size= 12,validation_data=Evalu_data,callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=5,
                            restore_best_weights=True),

                        learning_rate_reduction
                    ])

Epoch 1/27
26/26 [==============================] - 20s 587ms/step - loss: 0.2617 - accuracy: 0.8902 - val_loss: 0.0041 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 2/27
26/26 [==============================] - 13s 484ms/step - loss: 0.0186 - accuracy: 0.9928 - val_loss: 0.0251 - val_accuracy: 0.9942 - lr: 1.0000e-04
Epoch 3/27
26/26 [==============================] - 13s 490ms/step - loss: 0.0057 - accuracy: 0.9971 - val_loss: 8.8649e-05 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 4/27
26/26 [==============================] - 13s 486ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 0.0011 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 5/27
26/26 [==============================] - 13s 485ms/step - loss: 0.0226 - accuracy: 0.9913 - val_loss: 0.0126 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 6/27
26/26 [==============================] - 13s 483ms/step - loss: 0.0108 - accuracy: 0.9971 - val_loss: 1.1663e-05 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 7/27
26/26 [==============

In [ ]:
MobileNetV2_Model.save('/content/drive/MyDrive/3classnew/MobileNetV2_3c.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


**Testing the model**

In [ ]:
score = MobileNetV2_Model.evaluate(Test_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = MobileNetV2_Model.evaluate(Evalu_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 100.0 %
Accuracy for evaluation images: 100.0 %


Testing outside data set

In [ ]:
score = MobileNetV2_Model.evaluate(x= X_new, y = y_new, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = MobileNetV2_Model.evaluate(New_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 46.667 %
Accuracy for evaluation images: 44.762 %


In [ ]:
Resnet50_model.save('/content/drive/MyDrive/3classnew/Resnet50_3c.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
